In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install warnings
!pip install dgl
!pip install texttable

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Looking in indexes: https://pypi.org/simple

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
import random
from torchvision import transforms, datasets

import os
import pickle
from scipy.spatial.distance import cdist
from scipy import ndimage
import numpy as np

import dgl
import torch
import time
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
def sigma(dists, kth=8):
    # Get k-nearest neighbors for each node
    knns = np.partition(dists, kth, axis=-1)[:, kth::-1]

    # Compute sigma and reshape
    sigma = knns.sum(axis=1).reshape((knns.shape[0], 1))/kth
    return sigma + 1e-8 # adding epsilon to avoid zero value of sigma

def compute_adjacency_matrix_images(coord, feat, use_feat=False, kth=8):
    coord = coord.reshape(-1, 2)
    # Compute coordinate distance
    c_dist = cdist(coord, coord)

    if use_feat:
        # Compute feature distance
        f_dist = cdist(feat, feat)
        # Compute adjacency
        A = np.exp(- (c_dist/sigma(c_dist))**2 - (f_dist/sigma(f_dist))**2 )
    else:
        A = np.exp(- (c_dist/sigma(c_dist))**2)

    # Convert to symmetric matrix
    A = 0.5 * A * A.T
    A[np.diag_indices_from(A)] = 0
    return A

def compute_edges_list(A, kth=8+1):
    # Get k-similar neighbor indices for each node
    if 1==1:
        num_nodes = A.shape[0]
        new_kth = num_nodes - kth
        knns = np.argpartition(A, new_kth-1, axis=-1)[:, new_kth:-1]
        knns_d = np.partition(A, new_kth-1, axis=-1)[:, new_kth:-1]
    else:
        knns = np.argpartition(A, kth, axis=-1)[:, kth::-1]
        knns_d = np.partition(A, kth, axis=-1)[:, kth::-1]
    return knns, knns_d
class newCIFARSuperPix(torch.utils.data.Dataset):
    def __init__(self,
                 data_dir,
                 use_mean_px=True,
                 use_coord=True,
                 use_feat_for_graph_construct=False,):

        #self.split = split
        #self.is_test = split.lower() in ['test', 'val']
        with open(data_dir, 'rb') as f:
            self.labels, self.sp_data = pickle.load(f)

        self.use_mean_px = use_mean_px
        self.use_feat_for_graph = use_feat_for_graph_construct
        self.use_coord = use_coord
        self.n_samples = len(self.labels)
        self.img_size = 32

    def precompute_graph_images(self):
        #print('precompute all data for the %s set...' % self.split.upper())
        self.Adj_matrices, self.node_features, self.edges_lists = [], [], []
        for index, sample in enumerate(self.sp_data):
            mean_px, coord = sample[:2]
            coord = coord / self.img_size
            A = compute_adjacency_matrix_images(coord, mean_px, use_feat=self.use_feat_for_graph)
            edges_list, _ = compute_edges_list(A)
            N_nodes = A.shape[0]

            x = None
            if self.use_mean_px:
                x = mean_px.reshape(N_nodes, -1)
            if self.use_coord:
                coord = coord.reshape(N_nodes, 2)
                if self.use_mean_px:
                    x = np.concatenate((x, coord), axis=1)
                else:
                    x = coord
            if x is None:
                x = np.ones(N_nodes, 1)  # dummy features

            self.node_features.append(x)
            self.Adj_matrices.append(A)
            self.edges_lists.append(edges_list)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        g = dgl.DGLGraph()
        g.add_nodes(self.node_features[index].shape[0])
        g.ndata['feat'] = torch.Tensor(self.node_features[index])
        for src, dsts in enumerate(self.edges_lists[index]):
            g.add_edges(src, dsts[dsts!=src])

        return g, self.labels[index]

use_feat_for_graph_construct = False
tt = time.time()
data_with_feat_knn = newCIFARSuperPix("/content/drive/MyDrive/CMINST_data/CMNIST08_60000_75sp_train.pkl",use_feat_for_graph_construct=use_feat_for_graph_construct)

data_with_feat_knn.precompute_graph_images()
training_data = np.load('/content/drive/MyDrive/CMINST_data/colorMNIST08_60000_data.npy')
training_label=np.load('/content/drive/MyDrive/CMINST_data/colorMNIST08_60000_label.npy')

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import numpy as np
import os.path as osp
import pickle
import torch
import torch.utils
import torch.utils.data
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import InMemoryDataset,Data
#/content/drive/MyDrive/Colab_Notebooks/mnist08_83_75sp_train.pkl
def compute_adjacency_matrix_images(coord, sigma=0.1):
    coord = coord.reshape(-1, 2)
    dist = cdist(coord, coord)
    A = np.exp(- dist / (sigma * np.pi) ** 2)
    A[np.diag_indices_from(A)] = 0
    return A


def list_to_torch(data):
    for i in range(len(data)):
        if data[i] is None:
            continue
        elif isinstance(data[i], np.ndarray):
            if data[i].dtype == np.bool:
                data[i] = data[i].astype(np.float32)
            data[i] = torch.from_numpy(data[i]).float()
        elif isinstance(data[i], list):
            data[i] = list_to_torch(data[i])
    return data
def process(data_file):
  use_mean_px=True
  use_coord=True
  node_gt_att_threshold=0
  transform=None
  pre_transform=None
  pre_filter=None

  #data_file ='/content/drive/MyDrive/Colab_Notebooks/colorMNIST05_2000_75sp_train.pkl'

  with open(osp.join(data_file), 'rb') as f:
      labels,sp_data = pickle.load(f)

  #use_mean_px = self.use_mean_px
  #self.use_coord = self.use_coord
  n_samples = len(labels)
  img_size = 32
  #node_gt_att_threshold = self.node_gt_att_threshold

  edge_indices,xs,edge_attrs,node_gt_atts,edge_gt_atts = [], [], [], [], []
  data_list = []
  for index, sample in enumerate(sp_data):
      mean_px, coord = sample[:2]
      coord = coord / img_size
      A = compute_adjacency_matrix_images(coord)
      N_nodes = A.shape[0]

      A = torch.FloatTensor((A > 0.1) * A)
      edge_index, edge_attr = dense_to_sparse(A)

      x = None
      if use_mean_px:
          x = mean_px.reshape(N_nodes, -1)
      if use_coord:
          coord = coord.reshape(N_nodes, 2)
          if use_mean_px:
              x = np.concatenate((x, coord), axis=1)
          else:
              x = coord
      if x is None:
          x = np.ones(N_nodes, 1)  # dummy features

      # replicate features to make it possible to test on colored images
      x = np.pad(x, ((0, 0), (2, 0)), 'edge')
      if node_gt_att_threshold == 0:
          node_gt_att = (mean_px > 0).astype(np.float32)
      else:
          node_gt_att = mean_px.copy()
          node_gt_att[node_gt_att < node_gt_att_threshold] = 0

      node_gt_att = torch.LongTensor(node_gt_att).view(-1)
      row, col = edge_index
      edge_gt_att = torch.LongTensor(node_gt_att[row] * node_gt_att[col]).view(-1)

      data_list.append(
          Data(
              x=torch.tensor(x),
              y=torch.LongTensor([labels[index]]),
              edge_index=edge_index,
              edge_attr=edge_attr,
              node_gt_att=node_gt_att,
              edge_gt_att=edge_gt_att

          )
      )

  #torch.save(InMemoryDataset.collate(data_list), '/content/drive/MyDrive/Colab_Notebooks/colorMINST05_2000.pt')
  return data_list

In [ ]:
train_dir='/content/drive/MyDrive/CMINST_data/CMNIST095_10000_75sp_train.pkl'
val_dir='/content/drive/MyDrive/CMINST_data/CMNIST5000_75sp_val.pkl'
test_dir='/content/drive/MyDrive/CMINST_data/CMNIST10000_75sp_test.pkl'
training_final=process(data_file=train_dir)
valing_final=process(data_file=val_dir)
testing_final=process(data_file=test_dir)


In [ ]:
import torch
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch_geometric.nn.inits import glorot, zeros
import pdb

class GCNConv(MessagePassing):

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True,
                 edge_norm=True,
                 gfn=False):
        super(GCNConv, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.edge_norm = edge_norm
        self.gfn = gfn
        self.message_mask = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)

        edge_weight = edge_weight.view(-1)


        assert edge_weight.size(0) == edge_index.size(1)

        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        # Add edge_weight for loop edges.
        loop_weight = torch.full((num_nodes, ),
                                 1 if not improved else 2,
                                 dtype=edge_weight.dtype,
                                 device=edge_weight.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""

        x = torch.matmul(x, self.weight)
        if self.gfn:
            return x

        if not self.cached or self.cached_result is None:
            if self.edge_norm:
                edge_index, norm = GCNConv.norm(
                    edge_index,
                    x.size(0),
                    edge_weight,
                    self.improved,
                    x.dtype)
            else:
                norm = None
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):

        if self.edge_norm:
            return norm.view(-1, 1) * x_j
        else:
            return x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [ ]:
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Sequential, ReLU
from torch_geometric.nn import global_mean_pool, global_add_pool, GINConv, GATConv

import random
import pdb
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

layers=3
with_random=True
fc_num=222
hidden=32
eval_random=False
class causalpreCO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreCO, self).__init__()
    num_conv_layers = layers
    self.global_pool = global_add_pool

    self.with_random = with_random
    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)


    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num

    self.object_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )


    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias,0.0001)

  def forward(self,causal,edge_index,causal_edge_weight,batch,eval_random=True):


    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    #xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    #return xc_logis, xo_logis, xco_logis
    return self.objects_readout_layer(xo,batch)




  def objects_readout_layer(self,x,batch):
      xo = self.global_pool(x, batch)
      x_logis = F.log_softmax(self.object_mlp(xo), dim=-1)
      return x_logis



In [ ]:
class causalpreNO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreNO, self).__init__()
    num_conv_layers = layers

    self.global_pool = global_add_pool

    self.with_random = with_random

    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)

    self.context_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )

    self.random_readout_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    #else:
      #   assert False

    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0.0001)

  #def forward(self,causal,noncausal,batch,causal_edge_weight,noncausal_edge_weight,edge_index,eval_random=True):
  def forward(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random=True):

    xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index, noncausal_edge_weight))
    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    xc = self.global_pool(xc, batch)
    xo = self.global_pool(xo, batch)

    xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    return xc_logis, xco_logis


  def context_readout_layer(self, x):


      x_logis = F.log_softmax(self.context_mlp(x), dim=-1)
      return x_logis

  def random_readout_layer(self, xc, xo, eval_random):

      num = xc.shape[0]
      l = [i for i in range(num)]
      if self.with_random:
          if eval_random:
              random.shuffle(l)
      random_idx = torch.tensor(l)
      #if self.args.cat_or_add == "cat":
      #    x = torch.cat((xc[random_idx], xo), dim=1)
      #else:
      x = xc[random_idx] + xo

      x_logis = F.log_softmax(self.random_readout_mlp(x), dim=-1)
      return x_logis


In [ ]:
class CausalGAN(torch.nn.Module):
    """GCN with BN and residual connection."""
    def __init__(self, num_features,
                       num_classes,
                       gfn=False,
                       collapse=False,
                       residual=False,
                       res_branch="BNConvReLU",
                       global_pool="sum",
                       dropout=0.2,
                       edge_norm=True):
        super(CausalGAN, self).__init__()
        num_conv_layers = layers
        #hidden = args.hidden
        #self.args = args
        hidden=32
        head=4
        self.global_pool = global_add_pool
        self.dropout = dropout
        self.with_random = with_random
        self.without_node_attention = False
        self.without_edge_attention = False
        GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

        hidden_in = num_features
        self.num_classes = num_classes
        hidden_out = num_classes
        self.fc_num = fc_num
        self.bn_feat = BatchNorm1d(hidden_in)
        self.conv_feat = GCNConv(hidden_in, hidden, gfn=True) # linear transform
        self.bns_conv = torch.nn.ModuleList()
        self.convs = torch.nn.ModuleList()

        for i in range(num_conv_layers):
            self.bns_conv.append(BatchNorm1d(hidden))
            self.convs.append(GATConv(hidden, int(hidden / head), heads=head, dropout=dropout))

        self.edge_att_mlp = nn.Linear(hidden * 2, 2)
        self.node_att_mlp = nn.Linear(hidden, 2)
        self.bnc = BatchNorm1d(hidden)
        self.bno= BatchNorm1d(hidden)
        self.context_convs = GConv(hidden, hidden)
        self.objects_convs = GConv(hidden, hidden)
        self.relu = nn.ReLU(inplace=False)

        for m in self.modules():
            if isinstance(m, (torch.nn.BatchNorm1d)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0.0001)

    def forward(self, data, eval_random=True):

        x = data.x if data.x is not None else data.feat# x is the graph feature
        edge_index,edge_attr,batch = data.edge_index,data.edge_attr,data.batch
        row, col = edge_index
        x = self.bn_feat(x)
        x = self.relu(self.conv_feat(x, edge_index,edge_weight=edge_attr))

        for i, conv in enumerate(self.convs):
            x = self.bns_conv[i](x)
            x = self.relu(conv(x, edge_index))

        edge_rep = torch.cat([x[row], x[col]], dim=-1)

        if self.without_edge_attention:
            edge_att = 0.5 * torch.ones(edge_rep.shape[0], 2).to(device)
        else:
            edge_att = F.softmax(self.edge_att_mlp(edge_rep), dim=-1)
        edge_weight_c = edge_att[:, 0]
        edge_weight_o = edge_att[:, 1]

        if self.without_node_attention:
            node_att = 0.5 * torch.ones(x.shape[0], 2).to(device)
        else:
            node_att = F.softmax(self.node_att_mlp(x), dim=-1)
        xc = node_att[:, 0].view(-1, 1) * x
        xo = node_att[:, 1].view(-1, 1) * x

        #print(edge_weight_o)
        #xc = F.relu(self.context_convs(self.bnc(xc), edge_index, edge_weight_c))
        #xo = F.relu(self.objects_convs(self.bno(xo), edge_index, edge_weight_o))
        #node_of_graph=xo.shape[0]
        #causal_part,causal_node=split_graph(graph_x=xo,node_of_graph=node_of_graph)
        #noncausal_part,noncausal_node=split_graph(graph_x=xc,node_of_graph=node_of_graph,type_of_graph=False)

        #xc = self.global_pool(xc, batch)
        #xo = self.global_pool(xo, batch)

        #xc_logis = self.context_readout_layer(xc)
        #xo_logis = self.objects_readout_layer(xo)
        #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

      # return (xo,edge_index,edge_weight_o),(xc,edge_index,edge_weight_c),batch
        return (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch
        #return (causal_attention,causal_part,causal_node),(noncausal_attention,noncausal_part,noncausal_node),batch
        #return (xo,causal_part,causal_node),(xc,noncausal_part,noncausal_node),batch




In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR




In [ ]:
train_loader = DataLoader(training_final, batch_size=128, shuffle=True)
val_loader = DataLoader(valing_final, batch_size=128, shuffle=False)
test_loader = DataLoader(testing_final, batch_size=128, shuffle=False)
#t_load=[]
#for i in train_loader:
#  t_load.append(i)
#  if(len(t_load)==10000):
#    break


In [ ]:
number_of_class=10

causal_model2 = CausalGAN(7,number_of_class).to(device)
predictno_model2=causalpreNO(7,number_of_class).to(device)
predictco_model2=causalpreCO(7,number_of_class).to(device)
model_optimizer2 = torch.optim.Adam(
            list(causal_model2.parameters()) +
            list(predictno_model2.parameters())+list(predictco_model2.parameters()),
            lr=0.001)
Epo=500
lr_scheduler = CosineAnnealingLR(model_optimizer2, T_max=Epo, eta_min=1e-6, last_epoch=-1, verbose=False)

In [ ]:
predictno_model2

causalpreNO(
  (bnc): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bno): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (context_convs): GCNConv(32, 32)
  (objects_convs): GCNConv(32, 32)
  (context_mlp): Sequential(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
  (random_readout_mlp): Sequential(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [ ]:
causal_model2

CausalGAN(
  (bn_feat): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_feat): GCNConv(7, 32)
  (bns_conv): ModuleList(
    (0-2): 3 x BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (convs): ModuleList(
    (0-2): 3 x GATConv(32, 8, heads=4)
  )
  (edge_att_mlp): Linear(in_features=64, out_features=2, bias=True)
  (node_att_mlp): Linear(in_features=32, out_features=2, bias=True)
  (bnc): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bno): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (context_convs): GCNConv(32, 32)
  (objects_convs): GCNConv(32, 32)
  (relu): ReLU()
)

In [ ]:
predictco_model2

causalpreCO(
  (bnc): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bno): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (context_convs): GCNConv(32, 32)
  (objects_convs): GCNConv(32, 32)
  (object_mlp): Sequential(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [ ]:
import time
import json

loss_value=[]
loss_value_valation=[]
c=0.5
o=1
co=0.5
def num_graphs(data):
  if data.batch is not None:
      return data.num_graphs
  else:
      return data.x.size(0)
from tqdm import tqdm

for epoch in range(Epo):
  #model.train()
  causal_model2.train()
  predictno_model2.train()
  predictco_model2.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  correct_o = 0
  nb=0
  print(f"-----training-------{epoch}")
  loop = tqdm(enumerate(train_loader),total=len(train_loader))
  for it, data in loop:
#  for it, data in enumerate(train_loader):
      nb+=1
      model_optimizer2.zero_grad()
      data = data.to(device)

      one_hot_target = data.y.view(-1)
      #causal,noncausal,batch=causal_model(data)
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device)/number_of_class
     # sim=F.cosine_similarity(causal_attention,noncausal_attention).mean()
      #print(o_logs)
      #print(type(o_logs))
      c_loss = F.kl_div(c_logs, uniform_target,reduction='batchmean')
      #print(f'causal loss{c_loss}')
      o_loss = F.nll_loss(o_logs,one_hot_target)
      #print(f'noncausal loss{o_loss}')
      co_loss = F.nll_loss(co_logs,one_hot_target)
      #print(f'com causal loss{co_loss}')
      loss = c * c_loss + o * o_loss + co * co_loss

      pred_o = o_logs.max(1)[1]
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      loss.backward()
      total_loss += loss.item() #* num_graphs(data)
      total_loss_c += c_loss.item() #* num_graphs(data)
      total_loss_o += o_loss.item() #* num_graphs(data)
      total_loss_co += co_loss.item()# * num_graphs(data)
      model_optimizer2.step()
      loop.set_description(f"Epoch [{epoch}/{Epo}]")
      loop.set_postfix(loss = loss.item())

  #num = len(train_loader.dataset)
  lr_scheduler.step()
  total_loss = total_loss / nb
  total_loss_c = total_loss_c / nb
  print(f'number of {epoch} with total loss:{total_loss}')
  loss_value.append(total_loss)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    print(f"------valation---------{epoch}")
    causal_model2.eval()
    predictno_model2.eval()
    predictco_model2.eval()
    for data in val_loader:
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    print(f"causal val accuracy:{acc_co}")
    loss_value_valation.append(acc_co)
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary,indent=3)

    # Writing to sample.json
   # with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e08.json", "w") as outfile:
   #     outfile.write(json_object)

    torch.save({
            'causal_model.state_dic': causal_model2.state_dict(),
            'predictco_model_state_dict()': predictco_model2.state_dict(),
            'predictno_model_state_dict()': predictno_model2.state_dict(),
            'opt':model_optimizer2.state_dict()
            }, '/content/drive/MyDrive/running_cal_mnist/allmodel_cal_9533.pt')
    if(epoch>50):
      check=abs(acc_o-loss_value_valation[len(loss_value_valation)-20])/20
      if(check<=0.01):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()
  #model_optimizer.step()





-----training-------0


Epoch [0/500]: 100%|██████████| 79/79 [00:26<00:00,  2.97it/s, loss=0.0705]


number of 0 with total loss:0.3048795245682137
------valation---------0
causal val accuracy:0.1317
-----training-------1


Epoch [1/500]: 100%|██████████| 79/79 [00:26<00:00,  3.02it/s, loss=1.19]


number of 1 with total loss:0.314138225550893
------valation---------1
causal val accuracy:0.1325
-----training-------2


...

number of 49 with total loss:0.1514893762201448
------valation---------49
causal val accuracy:0.2608
-----training-------50


Epoch [50/500]: 100%|██████████| 79/79 [00:27<00:00,  2.92it/s, loss=0.0312]


number of 50 with total loss:0.14615903629720967
------valation---------50
causal val accuracy:0.2475
-----training-------51


Epoch [51/500]: 100%|██████████| 79/79 [00:26<00:00,  3.01it/s, loss=0.945]


number of 51 with total loss:0.1544845375833632
------valation---------51
causal val accuracy:0.2477


In [ ]:
sa a a

SyntaxError: ignored

In [ ]:
correct=0
correct_c=0
correct_o=0
print(f"------test---------{0000000}")
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
for data in test_loader:
  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
print(f"causal test accuracy:{acc_o}")
dictionary={"number of epoch":epoch,
            "training loss list":loss_value,
            "valation accuracy list":loss_value_valation,
            "test accuracy value":acc_co}

# Serializing json
json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e08.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
acc_co

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/running_cal_mnist/allmodel_cal_08.pt')
causal_model2.load_state_dict(checkpoint['causal_model.state_dic'])
predictco_model2.load_state_dict(checkpoint['predictco_model_state_dict()'])
predictno_model2.load_state_dict(checkpoint['predictno_model_state_dict()'])



In [ ]:
test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)

correct=0
correct_c=0
correct_o=0
print(f"------test---------{0000000}")
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
for data in test_loader:
  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
print(f"causal test accuracy:{acc_o}")
#dictionary={"number of epoch":epoch,
#            "training loss list":loss_value,
#            "valation accuracy list":loss_value_valation,
#            "test accuracy value":acc_co}

# Serializing json
#json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
#with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e08.json", "w") as outfile:
 #   outfile.write(json_object)


In [ ]:
aaa

In [ ]:
causal_model2 = CausalGCN(7,number_of_class).to(device)
predictco_model2=causalpreCO(7,number_of_class).to(device)

causal_model2.load_state_dict(torch.load('/content/drive/MyDrive/Colab_Notebooks/cau6000.pt'),strict=False)
causal_model2.eval()

predictco_model2.load_state_dict(torch.load('/content/drive/MyDrive/Colab_Notebooks/caupred6000.pt'),strict=False)
predictco_model2.eval()

In [ ]:
for i in range(len(loss_value)):
  loss_value[i]=float(loss_value[i])
plt.plot(np.array(loss_value))
plt.show()

In [ ]:

import json

for epoch in range(10):
  if epoch==3:
    break


# Data to be written
dictionary={"number of epoch":epoch}
# Serializing json
json_object = json.dumps(dictionary, indent=1)
# Writing to sample.json
with open("/content/drive/MyDrive/Colab_Notebooks/number_epoch.json", "w") as outfile:
    outfile.write(json_object)


In [ ]:
for i in parameters_to_prune:
  prune.remove(i[0], 'weight')

In [ ]:
for n, w in predictco_model.named_parameters():
  fix_para = {n: w < eps for n, w in predictco_model.named_parameters() if n.endswith('weight')}
  #print(f"weight:{sum(w)}")

  if n.endswith('weight'):
    print(w)
    print(n)




print(fix_para)

In [ ]:
plt.plot(np.array(loss_value))
plt.show()

In [ ]:
torch.zeros(32)

In [ ]:
t1=DataLoader(training_final, batch_size=1, shuffle=False)
t_load=[]
n=0
for i in t1:
  n+=1
  t_load.append(i)
n

In [ ]:
def mask_graph(graph_x,select_node):
  mask_value=np.array([0]*32)
  result=np.array([t.detach().numpy() for t in graph_x])
  for i in range(graph_x.shape[0]):
    if(i in select_node):
      continue
    result[i]=mask_value
  return torch.tensor(result)

def split_graph(graph_x,node_of_graph,type_of_graph=True):
  if(type_of_graph==True):
    select_node_number=int(node_of_graph/3)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    #print(select_node)
    return mask_graph(graph_x,select_node),select_node
  else:
    select_node_number=int(node_of_graph/3*2)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    return mask_graph(graph_x,select_node),select_node



def conf_u_node(dic,l):
  for i in l:
    if(i in dic.keys()):
      dic[i]+=1
      continue
    dic[i]=1
  return dic





In [ ]:
def mask_heter(heter_index,edweig,input_index,input_weight):
  edge_list2 = []
  for i in range(len(heter_index[0])):
    edge_list2.append((int(heter_index[0][i]),int(heter_index[1][i])))

  l1=[]
  l2=[]
  lw=[]
  for i in range(len(input_index[0])):
    if((int(input_index[0][i]),int(input_index[1][i])) in edge_list2):
      continue
    l1.append(input_index[0][i])
    l2.append(input_index[1][i])
    lw.append(input_weight[i])
  masked_index=torch.stack([torch.tensor(l1,dtype=torch.long),torch.tensor(l2,dtype=torch.long)]).to(device)
  masked_weight=torch.tensor(lw).to(device)
  return masked_index,masked_weight

In [ ]:
def myfunc(a, b):
  return a + b

x =map(myfunc, ('apple', 'banana', 'cherry'), ('orange', 'lemon', 'pineapple'))
list(x)

In [ ]:
homo_index[0]
homo_index[1]

In [ ]:
x = {"apple", "banana", "cherry"}
y = {"google", "microsoft", "apple","ww"}



print(y-x)

In [ ]:
index=torch.LongTensor([[],[]])
index1=torch.cat([index1,torch.tensor([[1],[0]])],dim=1)
torch.cat([index1,torch.tensor([[1],[0]])],dim=1)


In [ ]:
def dd(ed1,ed2,w):
  return (int(ed1),int(ed2),float(w))
def ff(l):
  index=torch.LongTensor([[],[]]).to(device)
  weight=torch.tensor([]).to(device)
  for i in l:
    index=torch.cat([index,torch.tensor([[i[0]],[i[1]]])],dim=1)
    weight=torch.cat([weight,torch.tensor([i[2]])])
  return index,weight
def mask_heter2(heter_index,edweig,input_index,input_weight):
  x=map(dd,heter_index[0],heter_index[1],edweig)
  x=set(x)
  #print(x)
  c=map(dd,input_index[0],input_index[1],input_weight)
  #list(c)
  c=set(c)
  f=list(c-x)
  return ff(f)
start = time.time()

mask_heter(homo_index,edweig_coa,edge_index,edge_weight_o)
end = time.time()
print(end - start)



In [ ]:
mask_heter2(homo_index,edweig_coa,edge_index,edge_weight_o)[0]

In [ ]:
(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)

In [ ]:
class learning_hetero(torch.nn.Module):
   def __init__(self,number_of_class=1):
     super(learning_hetero, self).__init__()
     self.mlp1 = nn.Linear(hidden*2, number_of_class)
     #self.lkrelu = nn.LeakyReLU()
     #self.mlp2 = nn.Linear(hidden, number_of_class)
     self.sigmod = nn.Sigmoid()

   def forward(self,graph,edge_index,batch):
     #edge_ho=self.get_label(data,batch)
     row, col = edge_index
     data=torch.cat([graph[row], graph[col]], dim=-1)
     x=self.mlp1(data)
     x=self.sigmod(x)
     return x


   def get_label(self,edge_index,graph,batch):
      edge_number=edge_index.shape[1]
      attr_mean=torch.mean(graph,dim=1)
      homo_threshold=torch.var(attr_mean)*3
      edge_classif=torch.zeros(edge_number)
      dic={}
      for i in range(len(attr_mean)):
        dic[int(i)]=attr_mean[i]

      for i in range(len(edge_classif)):
        if(abs(dic[int(edge_index[0][i])]-dic[int(edge_index[1][i])])>homo_threshold):
          edge_classif[i]=1
      return edge_classif

In [ ]:
model_heter=learning_hetero().to(device)
loss = nn.BCELoss()
hopt = torch.optim.Adam(model_heter.parameters(),lr=0.001)

In [ ]:
edge_index

In [ ]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
def k_hop_subgraph(
    node_idx,
    num_hops,
    edge_index,
    relabel_nodes = False,
    num_nodes = None,
    flow: str = 'source_to_target',
    directed: bool = False,):

    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    print(num_nodes)

    assert flow in ['source_to_target', 'target_to_source']
    if flow == 'target_to_source':
        row, col = edge_index
    else:
        col, row = edge_index

    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    for _ in range(num_hops):
        node_mask.fill_(False)
        node_mask[subsets[-1]] = True
        torch.index_select(node_mask, 0, row, out=edge_mask)
        subsets.append(col[edge_mask])

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True

    if not directed:
        edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]

    return subset, edge_index, inv, edge_mask

In [ ]:
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()

idex=4
tey=t_load[idex].to(device)
print(f"groud label:{tey.y}")
#print(tey)
#print(tey.batch)
tey.edge_index
(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)

In [ ]:

ground_heter_c=model_heter.get_label(edge_index,xc,batch)
het_c=model_heter(graph=xc,edge_index=edge_index,batch=batch)
#ground_heter_o=model_heter.get_label(edge_index,xo,batch)
het_o=model_heter(graph=xo,edge_index=edge_index,batch=batch)
fin_het=(het_c+het_o)/2
loss(fin_het,ground_heter_c.unsqueeze(1))
#loss_o=loss(het_c,ground_heter_o.unsqueeze(1))
edge_hetero_mask=fin_het.reshape(-1).round()


In [ ]:
torch.var(torch.mean(xo,dim=1))

In [ ]:
torch.mean(xo,dim=1)

In [ ]:
torch.var(torch.mean(xo,dim=1)*torch.mean(xo,dim=1))

In [ ]:
t1=torch.masked_select(tey.edge_index[0],ground_heter_c==1).to(device)
t2=torch.masked_select(tey.edge_index[1],ground_heter_c==1).to(device)
homo_index=torch.stack([t1,t2]).to(device)
edweig_coa=torch.masked_select(edge_weight_o,ground_heter_c==1).to(device)

In [ ]:
len(homo_index[0])

In [ ]:

start = time.time()
mask_heter(homo_index,edweig_coa,edge_index,edge_weight_o)
end = time.time()
print(end - start)

In [ ]:
from torch_geometric.utils import subgraph,k_hop_subgraph
def split_graph(graph_x,node_of_graph,edge_index,edge_weight,type_of_graph=True):
  if(type_of_graph==True):
    select_node_number=int(node_of_graph/3)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)

    #print(select_node)
    return mask_graph(graph_x,select_node),select_node,sedge_index,sedge_weight
  else:
    select_node_number=int(node_of_graph/2)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)
    return mask_graph(graph_x,select_node),select_node,sedge_index,sedge_weight
node_of_graph=xc.shape[0]
#select_node_number=int(node_of_graph/3)
#select_node=torch.topk(torch.mean(xo,dim=1),select_node_number)[1]
only_cau_part,cau_node,cau_index,cau_weight=split_graph(graph_x=xo,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_o,type_of_graph=True)
#subgraph(subset=select_node,edge_index=homo_index,edge_attr=edweig_coa)
#split_graph(graph_x=xc,node_of_graph=node_of_graph,edge_index=homo_index,edge_weight=edweig_coa,type_of_graph=True)

def mask_heter(homo_index,edweig,input_index,input_weight):
  edge_list2 = []
  for i in range(len(homo_index[0])):
    edge_list2.append((int(homo_index[0][i]),int(homo_index[1][i]),float(edweig[i])))
  homin=set(edge_list2)
  l1=[]
  l2=[]
  lw=[]
  for i in range(len(input_index[0])):
    if((int(input_index[0][i]),int(input_index[1][i]),float(input_weight[i])) in set(edge_list2)):
      l1.append(input_index[0][i])
      l2.append(input_index[1][i])
      lw.append(input_weight[i])
  masked_index=torch.stack([torch.tensor(l1,dtype=torch.long),torch.tensor(l2,dtype=torch.long)]).to(device)
  masked_weight=torch.tensor(lw).to(device)
  return masked_index,masked_weight
mask_heter(homo_index,edweig_coa,cau_index,cau_weight)


In [ ]:
subset,edge_indexk,inv,mask=k_hop_subgraph(0,1,tey.edge_index)
subgraph(subset,tey.edge_index,tey.edge_attr)[1]

In [ ]:
#from torch_geometric.utils import subgraph,k_hop_subgraph
#import random
#from numpy.random import choice
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()

idex=4
tey=t_load[idex].to(device)
print(f"groud label:{tey.y}")
#print(tey)
#print(tey.batch)

(xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(tey)
ground_heter_c=model_heter.get_label(edge_index,xc,batch)
het_c=model_heter(graph=xc,edge_index=edge_index,batch=batch)
#ground_heter_o=model_heter.get_label(edge_index,xo,batch)
het_o=model_heter(graph=xo,edge_index=edge_index,batch=batch)
fin_het=(het_c+het_o)/2
loss(fin_het,ground_heter_c.unsqueeze(1))
#loss_o=loss(het_c,ground_heter_o.unsqueeze(1))
edge_hetero_mask=fin_het.reshape(-1).round()
#c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
#v=uncertainty_mask(causal=causal_part,causal_node=causal_node,batch=batch,data=tey)[1]
#olog=predictco_model2.objects_readout_layer(x=v,batch=batch)
#olog=predictco_model(causal=causal_part,batch=batch)
def split_graph(graph_x,node_of_graph,edge_index,edge_weight,type_of_graph=True):
  if(type_of_graph==True):
    select_node_number=int(node_of_graph/3)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)

    #print(select_node)
    return mask_graph(graph_x,select_node),select_node,sedge_index,sedge_weight
  else:
    select_node_number=int(node_of_graph/3*2)
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)
    return mask_graph(graph_x,select_node),select_node,sedge_index,sedge_weight

def uncertainty_mask_gnerate(graph_x,node_in_graph,edge_index,edge_weight):
  all_mask=[]
  for i in node_in_graph:
    subset,edge_indexk,inv,mask=k_hop_subgraph(node_idx=int(i),num_hops=3,edge_index=edge_index)
    #random_mask=random.sample(node_in_graph.tolist(),int(len(node_in_graph)/3))
    sedge_index,sedge_weight=subgraph(subset=subset,edge_index=edge_index,edge_attr=edge_weight)
    all_mask.append((int(i),mask_graph(graph_x,subset),sedge_index,sedge_weight))
  return all_mask
node_of_graph=xo.shape[0]
g,node,index,weight=split_graph(graph_x=xo,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_o)
o_logs2=predictco_model2(causal=g,edge_index=index,causal_edge_weight=weight,batch=batch)

gd=edge_hetero_mask
t1=torch.masked_select(edge_index[0],gd==0)
t2=torch.masked_select(edge_index[1],gd==0)
homo_index=torch.stack([t1,t2])
edweig=torch.masked_select(edge_weight_o,gd==0)


print(o_logs.max(1)[1])
print(o_logs2.max(1)[1])
print(len(node))
print(len(torch.unique(homo_index)))
#k_hop_subgraph(node_idx=int(node[2]),num_hops=1,edge_index=edge_index)
#all=uncertainty_mask_gnerate(g,node,homo_index,edweig)
#all=uncertainty_mask_gnerate(g,node,index,weight)
#for n,i,j,k in all:
  #gd=model_heter.get_label(edge_index,xc,batch)
 # print(n)
 # o_logs=predictco_model2(causal=i,edge_index=j,causal_edge_weight=k,batch=batch)
 # print(F.nll_loss(o_logs,tey.y))
  #print(o_logs.max(1)[1])
#random_mask=random.sample(node_in_graph.tolist(),int(len(node_in_graph)/3))
#sedge_index,sedge_weight=subgraph(subset=subset,edge_index=edge_indexk,edge_attr=edge_weight)
#all_mask.append((mask_graph(graph_x,subset),sedge_index,sedge_weight))
#o_logs2=predictco_model2(causal=g,edge_index=index,causal_edge_weight=weight,batch=batch)
#olog=predictco_model2.objects_readout_layer(x=mask_graph(causal_attention,i),batch=batch)

#print(o_logs.max(1)[1])
#print(o_logs2.max(1)[1])
#  print(i)

#print(F.nll_loss(olog,tey.y))
#plt.hist(all_loss,bins=10)
#plt.show()
#print(all_logit)


In [ ]:
subset,edge_indexk,inv,mask=k_hop_subgraph(node_idx=2169,num_hops=3,edge_index=homo_index)

In [ ]:
gd=model_heter.get_label(edge_index,g,batch)
t1=torch.masked_select(edge_index[0],gd==0)
t2=torch.masked_select(edge_index[1],gd==0)
edweig=torch.masked_select(edge_weight_o,gd==0)

In [ ]:
homo_index=torch.stack([t1,t2])

In [ ]:
n,i,j,k=uncertainty_mask_gnerate(g,node,homo_index,edweig)[0]
o_logs=predictco_model2(causal=i,edge_index=j,causal_edge_weight=k,batch=batch)
print(F.nll_loss(o_logs,tey.y))
print(o_logs.max(1)[1])

def appendloss(batch_size,data,all_causallogit,logit):

  if(batch_size==1):
    all_causallogit.append(float(logit.view(-1)[data.y]))
  else:
    for i in range(batch_size):
      if(loss.max(1)[1][i]==data.y[i]):
        continue

      all_causallogit.append(float(logit[i,data.y[i]]))

In [ ]:
homo_index

In [ ]:
np.where(sp_data[0].mean(axis=1)==1)

In [ ]:
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from pylab import rcParams
def plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,causal):
    Y = squareform(pdist(sp_data[1], 'euclidean'))
    x_coord = sp_data[1] #np.flip(dataset.train.sp_data[_][1], 1)
    #intensities = sp_data[0].mean(axis=1)
    #label = classes[label]
    if real:
      intensities = sp_data[0].mean(axis=1)
    else:
      intensities=np.array(causal.mean(axis=1).cpu().detach())
      #intensities=np.array([0]*sp_data[0].shape[0])
      #intensities[[ 0,  3,  8, 14, 15, 24, 33, 37, 43, 45, 52]]=1




    G = nx.from_numpy_array(Y)
    pos = dict(zip(range(len(x_coord)), x_coord.tolist()))
    rotated_pos = {node: (y,-x) for (node, (x,y)) in pos.items()} # rotate the coords by 90 degree

    edge_list = []
    for src, dsts in enumerate(compute_edges_list(adj_matrix)[0]):
        for dst in dsts:
            edge_list.append((src, dst))

    nx.draw_networkx_nodes(G, rotated_pos, node_color=intensities, cmap=matplotlib.cm.Reds, node_size=60) #len(intensities))
    if with_edges:
        nx.draw_networkx_edges(G, rotated_pos, edge_list, alpha=0.3)

In [ ]:

plt.imshow(training_data_01[idex])
plt.show()
print("Label: ",training_label_01[idex])

In [ ]:
subset_all=[]
for i in range(5):
  sub_graph=choice(range(0,graph_nudes),number_of_node,p=F.softmax(tey.x.sum(axis=1)),replace=False)
  subset=torch.unique(subgraph(list(sub_graph),tey.edge_index,tey.edge_attr)[0])
  subset_all.append(subset)
subset_all

In [ ]:
edge_list2 = []

for i in range(len(homo_index[0])):
  edge_list2.append((int(homo_index[0][i]),int(homo_index[1][i])))


In [ ]:
sp_data=data_with_feat_knn.sp_data[idex]
adj_matrix=data_with_feat_knn.Adj_matrices[idex]
label=data_with_feat_knn[idex][1]
feat_coord=data_with_feat_knn.use_feat_for_graph
#causal_node=l
with_edges=True
plt.figure(figsize=(5, 5))
real=True
Y = squareform(pdist(sp_data[1], 'euclidean'))
x_coord = sp_data[1] #np.flip(dataset.train.sp_data[_][1], 1)
#intensities = sp_data[0].mean(axis=1)
#label = classes[label]
if real:
  intensities = sp_data[0].mean(axis=1)
else:
  intensities=np.array(g.mean(axis=1).cpu().detach())
  #intensities=np.array([0]*sp_data[0].shape[0])
  #intensities[[10]]=1




G = nx.from_numpy_array(Y)
pos = dict(zip(range(len(x_coord)), x_coord.tolist()))
rotated_pos = {node: (y,-x) for (node, (x,y)) in pos.items()} # rotate the coords by 90 degree

edge_list = []
for src, dsts in enumerate(compute_edges_list(adj_matrix)[0]):
    for dst in dsts:
        edge_list.append((src, dst))

nx.draw_networkx_nodes(G, rotated_pos, node_color=intensities, cmap=matplotlib.cm.Reds, node_size=60) #len(intensities))
if with_edges:
    nx.draw_networkx_edges(G, rotated_pos, edge_list2, alpha=0.3)


In [ ]:
edge_list

In [ ]:
edge_list2 = []

for i in range(len(homo_index[0])):
  edge_list2.append((int(homo_index[0][i]),int(homo_index[1][i])))


In [ ]:

sp_data=data_with_feat_knn.sp_data[idex]
adj_matrix=data_with_feat_knn.Adj_matrices[idex]
label=data_with_feat_knn[idex][1]
feat_coord=data_with_feat_knn.use_feat_for_graph
#causal_node=l
with_edges=True
plt.figure(figsize=(5, 5))
real=True
plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,tey.x)



In [ ]:
sp_data[0]

In [ ]:
plt.figure(figsize=(5, 5))
real=True
plot_superpixels_graph(sp_data, adj_matrix, label, feat_coord, with_edges,real,causal)

In [ ]:
plt.imshow(training_data_01[idex])
plt.show()
print("Label: ",training_label_01[idex])

In [ ]:
training_data_01[idex].shape